In [2]:
from openai import AzureOpenAI
import os

client = AzureOpenAI(
            azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
            api_version="2023-05-15"
        )

In [17]:
functions = [
    {
        "name": "get_customer_info",
        "description": "Get name list of IRM's customer",
        "parameters": {
            "type": "object",
            "properties": {
                "customer_name": {
                    "type": "string",
                    "description": "The name of the customer, e.g. Tapstry",
                },
            },
            "required": ["customer_name"],
        },
    },
    {
        "name": "create_review",
        "description": "Create a review for the IRM",
        "parameters": {
            "type": "object",
            "properties": {
                "review_text": {
                    "type": "string",
                    "description": "The text of the review, e.g. Great IRM!",
                },
            },
            "required": ["review_text"],
        },
    },
    {
        "name": "ask_vector_db",
        "description": "Ask any question related to IRM. This can include queries about administrator guide for settings.",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The question to ask, e.g. 'How to manage brands?'",
                },
            },
            "required": ["question"],
        },
    },
]

In [24]:
response = client.chat.completions.create(
            model="gpt-35-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "I am your assistant.",
                },
                {"role": "user", "content": "How to manage brands?"},
            ],
            functions=functions
        )
message = response.choices[0].message

In [25]:
print(message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"question":"How to manage brands?"}', name='ask_vector_db'), tool_calls=None)


In [30]:
import json
if message.function_call:
    print(message.function_call)
    function_name = message.function_call.name
    function_args_json = message.function_call.arguments
    function_args = json.loads(function_args_json)
    print(function_name)
    print(function_args)

FunctionCall(arguments='{"question":"How to manage brands?"}', name='ask_vector_db')
ask_vector_db
{'question': 'How to manage brands?'}
